# Load images with `tf.data`

## Setup

Start by [installing TensorFlow](https://www.tensorflow.org/install/).

And testing the installation:

In [0]:
import tensorflow as tf
tf.enable_eager_execution()
tf.VERSION

## Retrieve the images

Before you start any training, you'll need a set of images to teach the network about the new classes you want to recognize. We've created an archive of creative-commons licensed flower photos to use initially. 

In [0]:
import pathlib
data_root = tf.keras.utils.get_file('flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz', untar=True)
data_root = pathlib.Path(data_root)

After downloading 218MB, you should now have a copy of the flower photos available in your working directory.

In [0]:
for item in data_root.iterdir():
  print(item)

## Inspect the images
Now let's have a quick look at a couple of the images, so we know what we're dealing with:

In [0]:
attributions = (data_root/"LICENSE.txt").read_text().splitlines()[4:]
attributions = [line.split(' CC-BY') for line in attributions]
attributions = dict(attributions)

In [0]:
import IPython.display as display

def show_image(image_path):
    display.display(display.Image(str(image_path)))
    
    image_rel = pathlib.Path(image_path).relative_to(data_root)
    caption = "Image (CC BY 2.0) " + ' - '.join(attributions[str(image_rel)].split(' - ')[:-1])
    display.display(display.HTML("<div>%s</div>" % caption))

In [0]:
import random
all_image_paths = list(data_root.glob('*/*'))
random.shuffle(all_image_paths)

show_image(random.choice(all_image_paths))
show_image(random.choice(all_image_paths))
show_image(random.choice(all_image_paths))

## Determine the label for each image

List the available labels:

In [0]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

Assign an index to each label:

In [0]:
label_to_index = dict((name, index) for index,name in enumerate(label_names))
label_to_index

Create a list of every file, and its label index

In [0]:
all_image_labels = [label_to_index[path.parent.name] for path in all_image_paths]
all_image_labels[:10]

## A basic `tf.data.Dataset`

The easiest way to build a `tf.data.Dataset` is using the `from_tensor_slices` method.

Slicing the array of strings, results in a dataset of strings:

In [0]:
string_paths = [str(path) for path in all_image_paths]

path_ds = tf.data.Dataset.from_tensor_slices(string_paths)

The `output_shapes` and `output_types` fields describe the content of each item in the dataset. In this case it is a set of scalar binary-strings

In [0]:
print('shape: ', repr(path_ds.output_shapes))
print('type: ', path_ds.output_types)
print()
print(path_ds)

## A dataset of `(image, label)` pairs

Slicing both the paths and labels together gives a dataset of `(path, label)` pairs.

In [0]:
path_ds = tf.data.Dataset.from_tensor_slices((string_paths, all_image_labels))

In [0]:
for path,cls in path_ds.take(3):
  print(path.numpy(), " : ", label_names[cls.numpy()])
  print()

The `shapes` and `types`, are now tuples of shapes and types as well, describing each field:

In [0]:
print('shape: ', path_ds.output_shapes)
print('type: ', path_ds.output_types)
print()
print(path_ds)

## Load and format the images

Define a simple function to load, decode and format the image data.

In [0]:
def load_and_format_image(path, label):
  image = tf.read_file(path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize_images(image, [192, 192])
  image = (image/128.0) - 1

  return image, tf.cast(label, dtype=tf.int64)

Use the `map` method to convert the dataset of `(path,label)` pairs to `(image,label)` pairs.

Like all dataset methods, `map` does not execute the transformation immediately. It only executes as needed.

In [0]:
# many threads will be waiting on disk reads.
image_ds = path_ds.map(load_and_format_image, num_parallel_calls=32)
image_ds

In [0]:
import matplotlib.pyplot as plt

for image,label in image_ds.take(3):
  plt.figure()
  plt.imshow((image+1)/2)
  plt.title(label_names[label])
  plt.grid(False)

## Pipe to a model for training

To train a model with this dataset you will want the data:

* To be well shuffeled.
* To be batched.
* To repeat forever.
* Batches to available immediately when needed.

These features can be easily added using the `tf.data` api.

In [0]:
BATCH_SIZE = 4
# Shuffling the paths takes less memory than shuffling the images.
# Setting a buffer size larger than the dataset ensures that the data is completely shuffled.
ds = path_ds.shuffle(buffer_size=10000) 
ds = ds.map(load_and_format_image, num_parallel_calls=32)
ds = ds.batch(BATCH_SIZE).prefetch(1)
ds

### Quick Transfer learning with `keras.Applications`

In [0]:
mobile_net = tf.keras.applications.MobileNetV2(input_shape=[192, 192, 3], include_top=False)
mobile_net.trainable=False

model = tf.keras.Sequential([
  mobile_net,
  # This mobilnet returns a 6x6 feature map, take the spatial average.
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(len(label_names))
])

In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [0]:
model.summary()

In [0]:
len(model.trainable_variables) # Dense `weights` and `bias`

In [0]:
model.fit(ds, epochs=1, steps_per_epoch=3)  #steps_per_epoch=len(all_image_paths)//BATCH_SIZE